# **Desafio Final: Redes Neurais e Deep Learning**

# Objetivos

Exercitar os seguintes conceitos trabalhados no Curso:

*   Importação de datasets customizados.
*   Manipulação de dados em Python.
*   Interpretação de arquiteturas de redes neurais e transformação em código.
*   Criação de redes neurais em Keras.
*   Alteração de parâmetros de redes neurais e interpretação de resultados.

# Enunciado

Neste Desafio Final, nós iremos realizar a importação de um dataset público disponível no site Kaggle. O Kaggle é uma comunidade on-line de cientistas de dados e engenheiros de machine learning e deep learning, com diversos datasets disponíveis para utilização. Para utilizar algum dataset do Kaggle, precisamos seguir os seguintes passos:

1.   Criar uma conta gratuita no site https://www.kaggle.com/.
2.   Após criar a conta, logue no site do Kaggle e navegue até o dataset
https://www.kaggle.com/dhruvmak/face-mask-detection.
3.   Realize o download do dataset a ser utilizado (cerca de 200MB).
4.   Analise o dataset e os dados baixados.
5.   Realize o upload do arquivo para sua conta pessoal do Google Drive, a partir de onde iremos acessá-lo.

# Atividades

Os alunos deverão desempenhar as seguintes atividades:

1.   A partir dos exemplos de códigos mostrados durante o Bootcamp, carregar as amostras do dataset no Colab e separá-los nas suas respectivas classes, redimensionando-os para uma dimensionalidade de 256x256x3.
2.   Criar uma rede convolucional com a seguinte arquitetura:

*   Camada de entrada
*   Camada convolucional 1: 64 filtros, campo receptivo de tamanho 7x7.
*   Camada de pooling 1: janela de tamanho 2x2.
*   Camada convolucional 2: 128 filtros, campo receptivo de tamanho 5x5.
*   Camada de pooling 2: janela de tamanho 2x2.
*   Camada convolucional 3: 256 filtros, campo receptivo de tamanho 5x5.
*   Camada de pooling 3: janela de tamanho 2x2.
*   Camada convolucional 4: 512 filtros, campo receptivo de tamanho 3x3.
*   Camada convolucional 5: 1024 filtros, campo receptivo de tamanho 3x3.
*   Camada de pooling 4: janela de tamanho 2x2.
*   Camada de totalmente conectada 1: 256 neurônios.
*   Camada de totalmente conectada 2: 256 neurônios.
*   Camada de saída
*   Dropout com probabilidade de desativação de neurônios de 50%.

3.   Realizar a divisão das imagens em conjunto de treino (70%) e teste (30%),utilizando o método train_test_split do pacote Sklearn.
4.   Treinamentodaredecriadacom 30épocas,com um batch_sizede64,normalização dos valores de entrada, utilização do otimizador Adam e cálculo da métrica de acurácia.

In [7]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [8]:
# Caminho dos datasets
with_mask_folder = "/content/drive/MyDrive/Pessoal/Educação/Católica de Santa Catarina - Pós-Graduação/05 - Redes Neurais Artificiais e Deep Learning/CatolicaSC_RedesNeuraisArtificiaisDeepLearning_Entrega02/dataset/with_mask"
without_mask_folder = "/content/drive/MyDrive/Pessoal/Educação/Católica de Santa Catarina - Pós-Graduação/05 - Redes Neurais Artificiais e Deep Learning/CatolicaSC_RedesNeuraisArtificiaisDeepLearning_Entrega02/dataset/without_mask"

In [3]:
# Função para carregar e redimensionar as imagens
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (256, 256))
            images.append(img)
    return images

# Carregar imagens com máscara
with_mask_images = load_images_from_folder(with_mask_folder)
# Definir labels para imagens com máscara (1)
with_mask_labels = np.ones(len(with_mask_images))

# Carregar imagens sem máscara
without_mask_images = load_images_from_folder(without_mask_folder)
# Definir labels para imagens sem máscara (0)
without_mask_labels = np.zeros(len(without_mask_images))

# Concatenar todas as imagens e labels
images = np.array(with_mask_images + without_mask_images)
labels = np.concatenate((with_mask_labels, without_mask_labels))

# Verificar o formato dos dados
print("Shape das imagens:", images.shape)
print("Shape das labels:", labels.shape)

Shape das imagens: (440, 256, 256, 3)
Shape das labels: (440,)


In [9]:
# Inicializar o modelo
model = Sequential()

# Camadas convolucionais
model.add(Conv2D(64, (7, 7), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(256, (5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))

model.add(Conv2D(1024, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Camadas totalmente conectadas
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))

# Camada de saída
model.add(Dense(1, activation='sigmoid'))

# Dropout
model.add(Dropout(0.5))


In [10]:
# Early Stopping é uma técnica para evitar overfitting durante o treinamento.
# Ele monitora uma métrica (nesse caso, 'val_loss', perda no conjunto de validação),
# e interrompe o treinamento se a melhora não for observada após um certo número de épocas.
# Parâmetros:
# - monitor: a métrica a ser monitorada (neste caso, 'val_loss' - perda no conjunto de validação).
# - patience: número de épocas a esperar após a parada do progresso antes de parar o treinamento.
# - verbose: controle da saída de logs (0 para silencioso, 1 para detalhado).
# - mode: 'auto', 'min' ou 'max'. No modo 'auto', a direção da mudança é inferida automaticamente
#   com base na métrica monitorada. No modo 'min', o treinamento será interrompido quando a métrica
#   monitorada parar de diminuir. No modo 'max', o treinamento será interrompido quando a métrica
#   monitorada parar de aumentar.
# - baseline: valor de referência para a métrica monitorada. Se o desempenho da métrica não melhorar
#   além desse valor, o treinamento será interrompido.
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               verbose=0,
                               mode='auto',
                               baseline=None)

In [11]:
# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumo do modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 250, 250, 64)      9472      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 125, 125, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 121, 121, 128)     204928    
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 60, 60, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 56, 56, 256)       819456    
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 28, 28, 256)      

In [20]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)

# Verificar o formato dos dados de treino e teste
print("Shape dos dados de treino:", X_train.shape, y_train.shape)
print("Shape dos dados de teste:", X_test.shape, y_test.shape)

Shape dos dados de treino: (396, 256, 256, 3) (396,)
Shape dos dados de teste: (44, 256, 256, 3) (44,)


In [21]:
# Normalizar os valores de entrada
X_train = X_train / 255.0
X_test = X_test / 255.0

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/30
7/7 [==============================] - 19s 3s/step - loss: 7.6346 - accuracy: 0.5051 - val_loss: 8.4136 - val_accuracy: 0.4545
Epoch 2/30
7/7 [==============================] - 3s 395ms/step - loss: 7.6346 - accuracy: 0.5051 - val_loss: 8.4136 - val_accuracy: 0.4545
Epoch 3/30
7/7 [==============================] - 3s 383ms/step - loss: 7.6346 - accuracy: 0.5051 - val_loss: 8.4136 - val_accuracy: 0.4545
Epoch 4/30
7/7 [==============================] - 3s 388ms/step - loss: 7.6346 - accuracy: 0.5051 - val_loss: 8.4136 - val_accuracy: 0.4545
Epoch 5/30
7/7 [==============================] - 3s 385ms/step - loss: 7.6346 - accuracy: 0.5051 - val_loss: 8.4136 - val_accuracy: 0.4545
Epoch 6/30
7/7 [==============================] - 3s 393ms/step - loss: 7.6346 - accuracy: 0.5051 - val_loss: 8.4136 - val_accuracy: 0.4545


In [14]:
# Imprime a época em que o treinamento parou
print(early_stopping.stopped_epoch)

5


In [19]:
# Avaliação do modelo
model_accuracy = model.evaluate(X_test, y_test)
print("Precisão do modelo:", model_accuracy)

5/5 [==============================] - 0s 60ms/step - loss: 8.2968 - accuracy: 0.4621
Precisão do modelo: [8.2967529296875, 0.46212121844291687]
